In [0]:
import torch
import numpy as np
import torchvision
from torch import nn, optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [0]:
class MobileNet_V1(nn.Module):

    def __init__(self):
        super(MobileNet_V1, self).__init__()

        # 传统卷积层
        def conv_bn(input, output, stride):
            return nn.Sequential(
                nn.Conv2d(input, output, 3, stride, 1, bias=False),
                nn.BatchNorm2d(output),
                nn.ReLU()
            )
        
        # depthwise卷积层
        def conv_dw(input, output, stride):
            return nn.Sequential(
                nn.Conv2d(input, input, 3, stride, 1, groups=input, bias=False),
                nn.BatchNorm2d(input),
                nn.ReLU(),
                
                nn.Conv2d(input, output, 1, 1, 0, bias=False),
                nn.BatchNorm2d(output),
                nn.ReLU(),
            )
        
        self.model = nn.Sequential(
            conv_bn(3, 32, 2),
            conv_dw(32, 64, 1),
            conv_dw(64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            # nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.model(x)
        # print(x.shape)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

In [0]:
transform = transforms.Compose([
    # transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [27]:
train_data = torchvision.datasets.CIFAR10(root="./data", train=True,
                                          download=True, transform=transform)

test_data = torchvision.datasets.CIFAR10(root="./data", train=False,
                                          download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [28]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

x, y = iter(train_loader).next()
print("x:", x.shape, "y:", y.shape)

x: torch.Size([16, 3, 32, 32]) y: torch.Size([16])


In [0]:
# 类别关键字
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
#设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 模型
model = MobileNet_V1().to(device)
# print(model)
# print(x.shape, "x")
# print(model(x).shape, "o")
# 损失函数
criterion = nn.CrossEntropyLoss()

#优化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)



In [31]:
model.train()

for epoch in range(100):

    running_loss = 0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
     
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % 1000 == 0:
            print('[%d %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0

print("Done")

[1  1000] loss: 2.031
[1  2000] loss: 1.786
[1  3000] loss: 1.666
[2  1000] loss: 1.579
[2  2000] loss: 1.523
[2  3000] loss: 1.446
[3  1000] loss: 1.355
[3  2000] loss: 1.294
[3  3000] loss: 1.237
[4  1000] loss: 1.145
[4  2000] loss: 1.120
[4  3000] loss: 1.069
[5  1000] loss: 0.999
[5  2000] loss: 0.989
[5  3000] loss: 0.954
[6  1000] loss: 0.904
[6  2000] loss: 0.887
[6  3000] loss: 0.862
[7  1000] loss: 0.814
[7  2000] loss: 0.813
[7  3000] loss: 0.796
[8  1000] loss: 0.763
[8  2000] loss: 0.745
[8  3000] loss: 0.769
[9  1000] loss: 0.705
[9  2000] loss: 0.710
[9  3000] loss: 0.731
[10  1000] loss: 0.675
[10  2000] loss: 0.674
[10  3000] loss: 0.674
[11  1000] loss: 0.630
[11  2000] loss: 0.637
[11  3000] loss: 0.642
[12  1000] loss: 0.587
[12  2000] loss: 0.605
[12  3000] loss: 0.620
[13  1000] loss: 0.557
[13  2000] loss: 0.578
[13  3000] loss: 0.596
[14  1000] loss: 0.541
[14  2000] loss: 0.559
[14  3000] loss: 0.558
[15  1000] loss: 0.523
[15  2000] loss: 0.544
[15  3000] loss

In [36]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 79 %
